In [1]:
#Settings and imports
from settings.settings import data_settings

import pandas as pd
import matplotlib.pyplot as plt
import ta
pd.options.display.max_seq_items = 2000
pd.options.display.max_rows = 200

In [ ]:
import pandas

name = "EURUSD_H1_201801020000_202206210600"
csv_path = "D:/dev/workspace/vscode/python/bos/data/csv"
df=pd.read_csv(f"{csv_path}/with_class/{name}.csv")

df["DateTime"] = pd.to_datetime(df.pop("DateTime"))
# df = df.set_index(pd.DatetimeIndex(df["DateTime"]))



In [ ]:
df.columns

In [ ]:
import seaborn as sns
import matplotlib as mpl
mpl.rcParams["figure.figsize"]= (20,10)

sns.set_theme(style="darkgrid")
key = "buy_1"
print(df[df[key]==1].shape)

# g = sns.relplot(x="DateTime", y="Close", data=df,height=10,aspect=2,style="buy_1")#kind="line",
g=sns.scatterplot(data=df, x="DateTime", y="Close", hue=key)
g.figure.autofmt_xdate()


In [ ]:
#READ ALL DATA
# df = pd.read_csv(data_settings.formatted_data_path)
# df.pop("Volume")

# date_time = pd.to_datetime(df.pop("DateTime"),format=data_settings.date_time_format)
# df["DateTime"]= date_time

# df["ATR"]= ta.volatility.average_true_range(df["High"],df["Low"],df["Close"])
# df = df[13:].reset_index(drop=True)

In [ ]:
#SPLIT INTO PART
# df_parts = [
#             df[len(df)//8 * 0:len(df)//8 * 1 +290].reset_index(drop=True),
#             df[len(df)//8 * 1:len(df)//8 * 2 +290].reset_index(drop=True),
#             df[len(df)//8 * 2:len(df)//8 * 3 +290].reset_index(drop=True),
#             df[len(df)//8 * 3:len(df)//8 * 4 +290].reset_index(drop=True),
#             df[len(df)//8 * 4:len(df)//8 * 5 +290].reset_index(drop=True),
#             df[len(df)//8 * 5:len(df)//8 * 6 +290].reset_index(drop=True),
#             df[len(df)//8 * 6:len(df)//8 * 7 +290].reset_index(drop=True),
#             df[len(df)//8 * 7:len(df)//8 * 8 +290].reset_index(drop=True)
#             ]
# for index , df_part in enumerate(df_parts):
#     df_part.to_csv(data_settings.data_without_targets_parts_paths[index],index=False)

In [ ]:
#JOIN THE PARTS
# df_parts = [pd.read_csv(path) for path in data_settings.data_with_targets_parts_paths]
# df = pd.concat([part[:-290] for part in df_parts]).reset_index(drop=True)
# df.to_csv(data_settings.data_with_target_path,index=False)
# print(
# f"{len(df[df.buy_1==1])=}\n",
# f"{len(df[df.buy_2==1])=}\n",
# f"{len(df[df.buy_4==1])=}\n",
# f"{len(df[df.buy_8==1])=}\n",
# f"{len(df[df.buy_16==1])=}\n",
# f"{len(df[df.sell_1==1])=}\n",
# f"{len(df[df.sell_2==1])=}\n",
# f"{len(df[df.sell_4==1])=}\n",
# f"{len(df[df.sell_8==1])=}\n",
# f"{len(df[df.sell_16==1])=}\n",
# f"{len(df[df.do_nothing==1])=}\n",
# )

In [ ]:
# image_path = "D:/dev/workspace/vscode/python/bos/data/indicators/images/histograms/"
# for column in df.columns:
#     _=plt.hist(df[column],bins=50)
#     plt.savefig(f"{image_path}{column}.png")
    

In [ ]:
from concurrent.futures import ProcessPoolExecutor

from sklearn import datasets
from settings.settings import data_settings
import pandas as pd
import pathlib


def make_target(df: pd.DataFrame, target_path: str):

    window_size = data_settings.sl_window
    sl_to_atr = data_settings.sl_to_atr
    target_tp_to_sl_s = sorted(data_settings.target_tp_to_sl_s)

    # 0 --> sell 1          6  --> buy 1
    # 1 --> sell 1.25       7  --> buy 1.25
    # 2 --> sell 1.5        8  --> buy 1.5
    # 3 --> sell 1.75       9  --> buy 1.75
    # 4 --> sell 2          10 --> buy 2
    # 5 --> sell 4          11 --> buy 4

    buys = [[] for _ in target_tp_to_sl_s]
    sells = [[] for _ in target_tp_to_sl_s]

    # buy_0 = []
    # buy_1 = []
    # buy_2 = []
    # buy_3 = []
    # buy_4 = []
#
    # sell_0 = []
    # sell_1 = []
    # sell_2 = []
    # sell_3 = []
    # sell_4 = []

    do_nothing = []

    windowsizes = []

    for i in range(len(df)):

        sl = df.AVERAGE_TRUE_RANGE.iloc[i] * sl_to_atr

        do_nothing_found: int = 0

        buys_found = [0 for _ in target_tp_to_sl_s]

        buy_sl_hit: int = 0
        # buy_0_found: int = 0
        # buy_1_found: int = 0
        # buy_2_found: int = 0
        # buy_3_found: int = 0
        # buy_4_found: int = 0

        sells_found = [0 for _ in target_tp_to_sl_s]
        sell_sl_hit: int = 0
        # sell_0_found: int = 0
        # sell_1_found: int = 0
        # sell_2_found: int = 0
        # sell_3_found: int = 0
        # sell_4_found: int = 0

        bought = False
        sold = False
        n = len(target_tp_to_sl_s)
        for w in range(i+1, min(i+(window_size+1), len(df))):

            if(not buy_sl_hit and df.Low.iloc[w] <= df.Close.iloc[i] - sl):
                buy_sl_hit = 1

            else:
                for index, ratio in enumerate(target_tp_to_sl_s[::-1]):
                    if(not buys_found[-(index+1)] and not sold and df.High.iloc[w] >= df.Close.iloc[i] + ratio * sl):
                        buys_found[:(
                            n-index)] = [1 for _ in range(len(buys_found[:(n-index)]))]
                        bought = True
                        break
            # elif(not buy_4_found and not sold and df.High.iloc[w] >= df.Close.iloc[i] + 16*sl):
            #     buy_4_found = 1
            #     buy_3_found = 1
            #     buy_2_found = 1
            #     buy_1_found = 1
            #     buy_0_found = 1
            #     bought = True

            # elif(not buy_3_found and not sold and df.High.iloc[w] >= df.Close.iloc[i] + 8*sl):
            #     buy_3_found = 1
            #     buy_2_found = 1
            #     buy_1_found = 1
            #     buy_0_found = 1
            #     bought = True

            # elif(not buy_2_found and not sold and df.High.iloc[w] >= df.Close.iloc[i] + 4*sl):
            #     buy_2_found = 1
            #     buy_1_found = 1
            #     buy_0_found = 1
            #     bought = True

            # elif(not buy_1_found and not sold and df.High.iloc[w] >= df.Close.iloc[i] + 2*sl):
            #     buy_1_found = 1
            #     buy_0_found = 1
            #     bought = True

            # elif(not buy_0_found and not sold and df.High.iloc[w] >= df.Close.iloc[i] + 1*sl):
            #     buy_0_found = 1
            #     bought = True

            if(not sell_sl_hit and df.High.iloc[w] >= df.Close.iloc[i] + sl):
                sell_sl_hit = 1

            else:
                for index, ratio in enumerate(target_tp_to_sl_s[::-1]):
                    if(not sells_found[-(index+1)] and not bought and df.Low.iloc[w] <= df.Close.iloc[i] - ratio * sl):
                        sells_found[:(
                            n-index)] = [1 for _ in range(len(sells_found[:(n-index)]))]
                        bought = True
                        break

            # elif(not sell_4_found and not bought and df.Low.iloc[w] <= df.Close.iloc[i] - 16*sl):
            #     sell_4_found = 1
            #     sell_3_found = 1
            #     sell_2_found = 1
            #     sell_1_found = 1
            #     sell_0_found = 1
            #     sold = True

            # elif(not sell_3_found and not bought and df.Low.iloc[w] <= df.Close.iloc[i] - 8*sl):
            #     sell_3_found = 1
            #     sell_2_found = 1
            #     sell_1_found = 1
            #     sell_0_found = 1
            #     sold = True

            # elif(not sell_2_found and not bought and df.Low.iloc[w] <= df.Close.iloc[i] - 4*sl):
            #     sell_2_found = 1
            #     sell_1_found = 1
            #     sell_0_found = 1
            #     sold = True

            # elif(not sell_1_found and not bought and df.Low.iloc[w] <= df.Close.iloc[i] - 2*sl):
            #     sell_1_found = 1
            #     sell_0_found = 1
            #     sold = True

            # elif(not sell_0_found and not bought and df.Low.iloc[w] <= df.Close.iloc[i] - 1*sl):
            #     sell_0_found = 1
            #     sold = True

            if((buy_sl_hit and bought) or (sell_sl_hit and sold) or (buys_found[-1] or buys_found[-1])):
                break

        if(not bought and not sold):
            do_nothing_found = 1

        for _, values in enumerate(zip(buys, buys_found)):
            values[0].append(values[1])
        # buy_0.append(buy_0_found)
        # buy_1.append(buy_1_found)
        # buy_2.append(buy_2_found)
        # buy_3.append(buy_3_found)
        # buy_4.append(buy_4_found)

        # sell_0.append(sell_0_found)
        # sell_1.append(sell_1_found)
        # sell_2.append(sell_2_found)
        # sell_3.append(sell_3_found)
        # sell_4.append(sell_4_found)

        do_nothing.append(do_nothing_found)
        windowsizes.append(w-i)

        print(f"{i=}", "window size:", w-i)

    # df["buy_1"] = buy_0
    # df["buy_2"] = buy_1
    # df["buy_4"] = buy_2
    # df["buy_8"] = buy_3
    # df["buy_16"] = buy_4

    # df["sell_1"] = sell_0
    # df["sell_2"] = sell_1
    # df["sell_4"] = sell_2
    # df["sell_8"] = sell_3
    # df["sell_16"] = buy_4

    for _, values in enumerate(zip(buys, target_tp_to_sl_s)):
        df[f"buy_{values[1]}"] = values[0]

    for _, values in enumerate(zip(sells, target_tp_to_sl_s)):
        df[f"sell_{values[1]}"] = values[0]

    df["do_nothing"] = do_nothing
    df["windowsizes"] = windowsizes
    df.to_csv(target_path, index=False)
    return


def add_targets(data_name: str):

    window_size = data_settings.sl_window
    parts = 8

    csv_path = data_settings.csv_path
    cleaed_data_path = csv_path + "/with_indicator/" + data_name + ".csv"
    df = pd.read_csv(cleaed_data_path)

    n = df.shape[0] // parts
    dfs = [df[i * n:(i+1)*n + (window_size+1)].reset_index(drop=True)
           for i in range(parts)]

    target_paths = [
        f"D:/dev/workspace/vscode/python/bos/data/csv/with_class/{data_name}_part_1.csv",
        f"D:/dev/workspace/vscode/python/bos/data/csv/with_class/{data_name}_part_2.csv",
        f"D:/dev/workspace/vscode/python/bos/data/csv/with_class/{data_name}_part_3.csv",
        f"D:/dev/workspace/vscode/python/bos/data/csv/with_class/{data_name}_part_4.csv",
        f"D:/dev/workspace/vscode/python/bos/data/csv/with_class/{data_name}_part_5.csv",
        f"D:/dev/workspace/vscode/python/bos/data/csv/with_class/{data_name}_part_6.csv",
        f"D:/dev/workspace/vscode/python/bos/data/csv/with_class/{data_name}_part_7.csv",
        f"D:/dev/workspace/vscode/python/bos/data/csv/with_class/{data_name}_part_8.csv"
    ]

    with ProcessPoolExecutor() as executor:
        executor.map(make_target, dfs, target_paths)

    dfs = [pd.read_csv(target_path) for target_path in target_paths]

    df = pd.concat([part[:-(window_size+1)]
                   for part in dfs]).reset_index(drop=True)

    with_class_data_path = csv_path + "/with_class/" + \
        f"{data_name}_slatr_{data_settings.sl_to_atr:02}" + ".csv"

    df.to_csv(with_class_data_path, index=False)

    for path in target_paths:
        pathlib.Path(path).unlink(missing_ok=True)


if __name__ == "__main__":
    pass
